# 1. 라이브러리 

In [1]:
# 데이터 처리 라이브러리
import os
import os.path as pth
from tqdm import tqdm
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import cv2
import mediapipe as mp

# Tensorflow 관련 라이브러리
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, BatchNormalization, Flatten, Activation, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=15000)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)


# 2. 데이터 불러오기

## 2-1. 필요한 라벨링만 불러오기

In [2]:
label_df = pd.read_excel('./data/categries_emo.xlsx')
label_dict = dict(label_df[['Num','Emotion']].values)

# 파일 제목에서 3번째 항목만 사용
print(label_df)

emotion_arr = label_df.loc[:,'Num'].values


emotion_list = emotion_arr.tolist()
print(emotion_list)
    

   Num    Emotion
0    1    neutral
1    2       calm
2    3      happy
3    4        sad
4    5      angry
5    6    fearful
6    7    disgust
7    8  surprised
[1, 2, 3, 4, 5, 6, 7, 8]


## 2-2. 경로 불러오기

In [3]:
# 파일의 경로와 라벨정보를 담은 df 생성
path = './data/emotion_data'

files = list()
categories = list()


for img_cat in os.listdir(path):
    cat = path + '/' + img_cat
    
    for filename in os.listdir(cat):
        file_list = cat + '/' + filename
        files.append(file_list)
        
        emo = filename[7]
#        print(emo)
        
        for n in emo:
            categories.append(n)


train_data = pd.DataFrame({'file':files,
                           'label':categories})


train_data
        
        

,file,label
0,./data/emotion_data/Actor_12/02-01-06-01-01-02...,6
1,./data/emotion_data/Actor_12/02-01-08-01-01-01...,8
2,./data/emotion_data/Actor_12/02-01-08-01-02-02...,8
3,./data/emotion_data/Actor_12/02-01-07-02-01-02...,7
4,./data/emotion_data/Actor_12/02-01-07-01-02-01...,7
...,...,...
1436,./data/emotion_data/Actor_14/02-01-01-01-02-01...,1
1437,./data/emotion_data/Actor_14/02-01-07-01-01-02...,7
1438,./data/emotion_data/Actor_14/02-01-03-02-01-01...,3
1439,./data/emotion_data/Actor_14/02-01-08-01-02-02...,8


## 2-3. 동영상 프레임 단위로 변환

In [4]:
# 변환 끝

for file in files:
    vidcap = cv2.VideoCapture(file)
    success,image = vidcap.read()

    count = 1
    success = True
    
    labeled_file = file.split('/')[-1].split('.')[0]
    
    while success:
       
        success,image = vidcap.read()
        if success == False:
            break
        imWrite=cv2.imwrite('./data/emotion_img/%s_%d.jpg' %(labeled_file,count), image)        
        if imWrite==True:
            print("Success Saved Image")
           
        else:
            print("Faile Saved Image")
            
       
        print('%s_%d.jpg' %(labeled_file,count))
        
        if cv2.waitKey(10) == 27:
            break
        count += 1
         
        

# 이미지 몇개인지 테스트 해봄        
        
        
#         if success ==True:
#             #cv2_imshow(image)
#             #break
#             count+=1

# print(count)
    
    

Success Saved Image
02-01-06-01-01-02-12_1.jpg
Success Saved Image
02-01-06-01-01-02-12_2.jpg
Success Saved Image
02-01-06-01-01-02-12_3.jpg
Success Saved Image
02-01-06-01-01-02-12_4.jpg
Success Saved Image
02-01-06-01-01-02-12_5.jpg
Success Saved Image
02-01-06-01-01-02-12_6.jpg
Success Saved Image
02-01-06-01-01-02-12_7.jpg
Success Saved Image
02-01-06-01-01-02-12_8.jpg
Success Saved Image
02-01-06-01-01-02-12_9.jpg
Success Saved Image
02-01-06-01-01-02-12_10.jpg
Success Saved Image
02-01-06-01-01-02-12_11.jpg
Success Saved Image
02-01-06-01-01-02-12_12.jpg
Success Saved Image
02-01-06-01-01-02-12_13.jpg
Success Saved Image
02-01-06-01-01-02-12_14.jpg
Success Saved Image
02-01-06-01-01-02-12_15.jpg
Success Saved Image
02-01-06-01-01-02-12_16.jpg
Success Saved Image
02-01-06-01-01-02-12_17.jpg
Success Saved Image
02-01-06-01-01-02-12_18.jpg
Success Saved Image
02-01-06-01-01-02-12_19.jpg
Success Saved Image
02-01-06-01-01-02-12_20.jpg
Success Saved Image
02-01-06-01-01-02-12_21.jpg
S

Success Saved Image
02-01-08-01-01-01-12_74.jpg
Success Saved Image
02-01-08-01-01-01-12_75.jpg
Success Saved Image
02-01-08-01-01-01-12_76.jpg
Success Saved Image
02-01-08-01-01-01-12_77.jpg
Success Saved Image
02-01-08-01-01-01-12_78.jpg
Success Saved Image
02-01-08-01-01-01-12_79.jpg
Success Saved Image
02-01-08-01-01-01-12_80.jpg
Success Saved Image
02-01-08-01-01-01-12_81.jpg
Success Saved Image
02-01-08-01-01-01-12_82.jpg
Success Saved Image
02-01-08-01-01-01-12_83.jpg
Success Saved Image
02-01-08-01-01-01-12_84.jpg
Success Saved Image
02-01-08-01-01-01-12_85.jpg
Success Saved Image
02-01-08-01-01-01-12_86.jpg
Success Saved Image
02-01-08-01-01-01-12_87.jpg
Success Saved Image
02-01-08-01-01-01-12_88.jpg
Success Saved Image
02-01-08-01-01-01-12_89.jpg
Success Saved Image
02-01-08-01-01-01-12_90.jpg
Success Saved Image
02-01-08-01-01-01-12_91.jpg
Success Saved Image
02-01-08-01-01-01-12_92.jpg
Success Saved Image
02-01-08-01-01-01-12_93.jpg
Success Saved Image
02-01-08-01-01-01-12

Success Saved Image
02-01-07-02-01-02-12_41.jpg
Success Saved Image
02-01-07-02-01-02-12_42.jpg
Success Saved Image
02-01-07-02-01-02-12_43.jpg
Success Saved Image
02-01-07-02-01-02-12_44.jpg
Success Saved Image
02-01-07-02-01-02-12_45.jpg
Success Saved Image
02-01-07-02-01-02-12_46.jpg
Success Saved Image
02-01-07-02-01-02-12_47.jpg
Success Saved Image
02-01-07-02-01-02-12_48.jpg
Success Saved Image
02-01-07-02-01-02-12_49.jpg
Success Saved Image
02-01-07-02-01-02-12_50.jpg
Success Saved Image
02-01-07-02-01-02-12_51.jpg
Success Saved Image
02-01-07-02-01-02-12_52.jpg
Success Saved Image
02-01-07-02-01-02-12_53.jpg
Success Saved Image
02-01-07-02-01-02-12_54.jpg
Success Saved Image
02-01-07-02-01-02-12_55.jpg
Success Saved Image
02-01-07-02-01-02-12_56.jpg
Success Saved Image
02-01-07-02-01-02-12_57.jpg
Success Saved Image
02-01-07-02-01-02-12_58.jpg
Success Saved Image
02-01-07-02-01-02-12_59.jpg
Success Saved Image
02-01-07-02-01-02-12_60.jpg
Success Saved Image
02-01-07-02-01-02-12

Success Saved Image
02-01-07-01-02-01-12_85.jpg
Success Saved Image
02-01-07-01-02-01-12_86.jpg
Success Saved Image
02-01-07-01-02-01-12_87.jpg
Success Saved Image
02-01-07-01-02-01-12_88.jpg
Success Saved Image
02-01-07-01-02-01-12_89.jpg
Success Saved Image
02-01-07-01-02-01-12_90.jpg
Success Saved Image
02-01-07-01-02-01-12_91.jpg
Success Saved Image
02-01-07-01-02-01-12_92.jpg
Success Saved Image
02-01-07-01-02-01-12_93.jpg
Success Saved Image
02-01-07-01-02-01-12_94.jpg
Success Saved Image
02-01-07-01-02-01-12_95.jpg
Success Saved Image
02-01-07-01-02-01-12_96.jpg
Success Saved Image
02-01-07-01-02-01-12_97.jpg
Success Saved Image
02-01-07-01-02-01-12_98.jpg
Success Saved Image
02-01-07-01-02-01-12_99.jpg
Success Saved Image
02-01-07-01-02-01-12_100.jpg
Success Saved Image
02-01-07-01-02-01-12_101.jpg
Success Saved Image
02-01-07-01-02-01-12_102.jpg
Success Saved Image
02-01-07-01-02-01-12_103.jpg
Success Saved Image
02-01-07-01-02-01-12_104.jpg
Success Saved Image
02-01-07-01-02-

Success Saved Image
02-01-05-01-02-01-12_49.jpg
Success Saved Image
02-01-05-01-02-01-12_50.jpg
Success Saved Image
02-01-05-01-02-01-12_51.jpg
Success Saved Image
02-01-05-01-02-01-12_52.jpg
Success Saved Image
02-01-05-01-02-01-12_53.jpg
Success Saved Image
02-01-05-01-02-01-12_54.jpg
Success Saved Image
02-01-05-01-02-01-12_55.jpg
Success Saved Image
02-01-05-01-02-01-12_56.jpg
Success Saved Image
02-01-05-01-02-01-12_57.jpg
Success Saved Image
02-01-05-01-02-01-12_58.jpg
Success Saved Image
02-01-05-01-02-01-12_59.jpg
Success Saved Image
02-01-05-01-02-01-12_60.jpg
Success Saved Image
02-01-05-01-02-01-12_61.jpg
Success Saved Image
02-01-05-01-02-01-12_62.jpg
Success Saved Image
02-01-05-01-02-01-12_63.jpg
Success Saved Image
02-01-05-01-02-01-12_64.jpg
Success Saved Image
02-01-05-01-02-01-12_65.jpg
Success Saved Image
02-01-05-01-02-01-12_66.jpg
Success Saved Image
02-01-05-01-02-01-12_67.jpg
Success Saved Image
02-01-05-01-02-01-12_68.jpg
Success Saved Image
02-01-05-01-02-01-12

KeyboardInterrupt: 

In [6]:
# 이미지 리스트 만들기

img_list = list()
img_path = './data/emotion_img'


for img in os.listdir(img_path):
    file_path = img_path + '/' + img
    img_list.append(file_path)
    
# print(img_list)
    


# 3. TFRecord 데이터 처리

## 3-1. TFRecord 압축파일 생성

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _floatarray_feature(array):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=array))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _validate_text(text):
    """If text is not str or unicode, then try to convert it to str."""
    if isinstance(text, str):
        return text
    elif isinstance(text, 'unicode'):
        return text.encode('utf8', 'ignore')
    else:
        return str(text)

In [ ]:
def to_tfrecords(id_list, randmark_id_list, tfrecords_name):
    print("Start converting")
    options = tf.io.TFRecordOptions(compression_type = 'GZIP')
    with tf.io.TFRecordWriter(path=pth.join(tfrecords_name+'.tfrecords'), options=options) as writer:
        for id_, randmark_id in tqdm(zip(id_list, randmark_id_list), total=len(id_list), position=0, leave=True):
            image_path = pth.join(train_dirs, id_)
            _binary_image = tf.io.read_file(image_path)

            string_set = tf.train.Example(features=tf.train.Features(feature={
                'image_raw': _bytes_feature(_binary_image),
                'randmark_id': _int64_feature(randmark_id),
                'id': _bytes_feature(id_.encode()),
            }))

            writer.write(string_set.SerializeToString())    

In [ ]:
with tf.device('/device:GPU:1'):
    train_ids, val_ids, train_landmark_ids, val_landmark_ids = train_test_split(train_data['file'], train_data['label'], test_size=0.2, random_state=42, shuffle=True,
                                                                                stratify=train_data['label'])

    to_tfrecords(train_ids, train_landmark_ids, pth.join(path, 'tf_record_train'))
    to_tfrecords(val_ids, val_landmark_ids, pth.join(path, 'tf_record_valid'))

# 3. Face Mesh 

In [4]:
# cv2.imshow를 대신하는 함수
# 반드시 먼저 실행

def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    return plt.imshow(a, **kwargs)

In [7]:
test_list = img_list[0:9]

print(test_list)

labeled_test_list = test_list[1].split('/')[-1].split('.')[0]
labeled_test_list

['./data/emotion_img/02-01-07-02-01-02-12_98.jpg', './data/emotion_img/02-01-06-01-01-02-12_103.jpg', './data/emotion_img/02-01-08-01-02-02-12_18.jpg', './data/emotion_img/02-01-06-01-01-02-12_69.jpg', './data/emotion_img/02-01-08-01-01-01-12_92.jpg', './data/emotion_img/02-01-08-01-02-02-12_70.jpg', './data/emotion_img/02-01-08-01-02-02-12_95.jpg', './data/emotion_img/02-01-06-01-01-02-12_7.jpg', './data/emotion_img/02-01-04-02-02-01-12_19.jpg']


'02-01-06-01-01-02-12_103'

In [64]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh


# 이미지 처리
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True,
                                  max_num_faces=1,
                                  min_detection_confidence=0.5)


drawing_spec = mp_drawing.DrawingSpec(thickness=1, 
                                      circle_radius=1)

test_list = img_list[0:5]


# 위에서 만든 jpg 리스트 넣기
for idx, img in enumerate(test_list):
    image = cv2.imread(img)
    image_conv=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_conv)
    
    labeled_test_list = img.split('/')[-1].split('.')[0]
            
    # 테스트
    if not results.multi_face_landmarks:
        continue
    annotated_image = image.copy()
    
    fileTemp=''
    
    for face_landmarks in results.multi_face_landmarks:
        print('face_landmarks:', face_landmarks)
      
        
        
        fileTemp+=str(face_landmarks)
        
        
        mp_drawing.draw_landmarks(image=annotated_image,
                                  landmark_list=face_landmarks,
                                  connections=mp_face_mesh.FACE_CONNECTIONS,
                                  landmark_drawing_spec=drawing_spec,
                                  connection_drawing_spec=drawing_spec)
    cv2.imwrite('./data/emotion_land/%s.png' %labeled_test_list, annotated_image)
    with open('./data/emotion_land/%s.txt'%labeled_test_list,'w') as f:
        f.write(str(fileTemp))
        
face_mesh.close()
        

    
    
# 위에서 만든 jpg 리스트 넣기
# for idx, img in enumerate(img_list):
#     image = cv2.imread(img)
#     image_conv=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
#     results = face_mesh.process(image_conv)    
    
    

#     # 랜드마크 찍기
#     if not results.multi_face_landmarks:
#         continue
#     annotated_image = image.copy()

#     for face_landmarks in results.multi_face_landmarks:
#         print('face_landmarks:', face_landmarks)
        
#         mp_drawing.draw_landmarks(image=annotated_image,
#                                   landmark_list=face_landmarks,
#                                   connections=mp_face_mesh.FACE_CONNECTIONS,
#                                   landmark_drawing_spec=drawing_spec,
#                                   connection_drawing_spec=drawing_spec)
#     cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', image)

# face_mesh.close()

face_landmarks: landmark {
  x: 0.49957600235939026
  y: 0.7179313898086548
  z: -0.023241404443979263
  visibility: 0.0
  presence: 0.0
}
landmark {
  x: 0.49139684438705444
  y: 0.660869300365448
  z: -0.05580984428524971
  visibility: 0.0
  presence: 0.0
}
landmark {
  x: 0.49544256925582886
  y: 0.6738917827606201
  z: -0.02634384296834469
  visibility: 0.0
  presence: 0.0
}
landmark {
  x: 0.47905638813972473
  y: 0.596677839756012
  z: -0.046401891857385635
  visibility: 0.0
  presence: 0.0
}
landmark {
  x: 0.48956799507141113
  y: 0.6424808502197266
  z: -0.06078396365046501
  visibility: 0.0
  presence: 0.0
}
landmark {
  x: 0.488029420375824
  y: 0.6157609820365906
  z: -0.05864952877163887
  visibility: 0.0
  presence: 0.0
}
landmark {
  x: 0.48523566126823425
  y: 0.5467630624771118
  z: -0.03723722696304321
  visibility: 0.0
  presence: 0.0
}
landmark {
  x: 0.4162484109401703
  y: 0.5439925193786621
  z: 0.009622330777347088
  visibility: 0.0
  presence: 0.0
}
landmark {


In [63]:
strstr=''
with open('./data/emotion_land/02-01-08-01-02-02-12_18.txt','r') as f:
    strstr=f.read()

bytearr=bytearray(strstr,'utf-8')
print(bytearr)

bytearray(b'bytearray(b\'\\n\\x19\\r\\xd6\\xef\\x02?\\x15\\x8a\\xd6.?\\x1d\\x08\\xaa\\xc9\\xbc%\\x00\\x00\\x00\\x00-\\x00\\x00\\x00\\x00\\n\\x19\\r\\xab\\xd3\\x01?\\x15\\xb8\\x98\\x1e?\\x1d\\x91Le\\xbd%\\x00\\x00\\x00\\x00-\\x00\\x00\\x00\\x00\\n\\x19\\r|\\xad\\x01?\\x15T\\x01#?\\x1d\\r\\xda\\xdb\\xbc%\\x00\\x00\\x00\\x00-\\x00\\x00\\x00\\x00\\n\\x19\\r\\xfaX\\xfa>\\x15#S\\r?\\x1d\\x91\\xee?\\xbd%\\x00\\x00\\x00\\x00-\\x00\\x00\\x00\\x00\\n\\x19\\r\\x15V\\x01?\\x15NS\\x19?\\x1d\\x85Ay\\xbd%\\x00\\x00\\x00\\x00-\\x00\\x00\\x00\\x00\\n\\x19\\r5b\\x00?\\x15\\x7f\\x00\\x12?\\x1d6#n\\xbd%\\x00\\x00\\x00\\x00-\\x00\\x00\\x00\\x00\\n\\x19\\ruo\\xfb>\\x15\\x1a@\\xff>\\x1d\\xdf\\xbd\\n\\xbd%\\x00\\x00\\x00\\x00-\\x00\\x00\\x00\\x00\\n\\x19\\r\\x06\\x1c\\xd1>\\x15\\xbf\\xc8\\x02?\\x1d\\xaa5m8%\\x00\\x00\\x00\\x00-\\x00\\x00\\x00\\x00\\n\\x19\\r\\xe5\\x1e\\xf8>\\x15R\\xb0\\xe2>\\x1d9\\x14\\xf4\\xbc%\\x00\\x00\\x00\\x00-\\x00\\x00\\x00\\x00\\n\\x19\\ra\\xb1\\xf6>\\x15\\xb2\\x11\\xd4>\\x1d\\x01\\xc